#Lab.00 / IBM3202 - Installing Software on Google Colab for IBM3202 tutorials

While most software and applications required for the IBM3202 Molecular Modeling and Simulation course can be installed into Google Colab during the tutorials, some other software takes a significant amount of time between compilation and installation. To avoid consuming the limited time of the tutorials for these tasks, here we include the installation step for such programs.

The most common procedure comprises:
1. Compilation of the required software during a Google Colab session
2. Backing up the compiled software into your Google Drive
3. By the time the tutorials take place, these precompiled programs will be onto your Google Colab session

#Before we start

##**Copy this notebook to your Drive, otherwise the installation will not work**

You can do this by clicking on the **Save a Copy to Drive** option on the **File** menu.

#Installation of pyRosetta

##Getting the PyRosetta License

First enter [the following link](https://els2.comotion.uw.edu/product/pyrosetta), fill the required information and press the buttons indicated by the black arrows to obtain a pyRosetta License

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/pyrosetta_01.png'/>
</center>
</figure>

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/pyrosetta_02.png'/>
</center>
</figure>

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/pyrosetta_03.png'/>
</center>
</figure>

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/pyrosetta_04.png'/>
</center>
</figure>

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/pyrosetta_05.png'/>
</center>
</figure>

At this point, you will obtain a **username and password** for obtaining pyRosetta.
<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/pyrosetta_06.png'/>
</center>
</figure>

##Setting up Pyrosetta in Google Colab

To install pyRosetta, we will mount your Google Drive to be able to transfer files. You will be requested to authorize this Google Drive mounting by entering an authorization code into an input box. Copy the code, paste it into this box and then press 'Enter'.

In [ ]:
# Mounting Google Drive and add it to Python sys path  

google_drive_mount_point = '/content/google_drive'

import os, sys, time

if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount(google_drive_mount_point)

google_drive = google_drive_mount_point + '/My Drive'
google_drive_prefix = google_drive + '/prefix'

if not os.path.isdir(google_drive_prefix):
    os.mkdir(google_drive_prefix)

pyrosetta_install_prefix_path = '/content/prefix'
if os.path.islink(pyrosetta_install_prefix_path):
    os.unlink(pyrosetta_install_prefix_path)
os.symlink(google_drive_prefix, pyrosetta_install_prefix_path)

sys.path.insert(0, pyrosetta_install_prefix_path)

In [ ]:
os.environ["PYTHONPATH"] = os.environ["PYTHONPATH"] + ':' + pyrosetta_install_prefix_path

In the following script, we will compile and intall pyRosetta, and also back it up to Google Drive for future usage.

Please note that you have to **replace --user and --password** in this script with the one obtained through your pyRosetta license

In [ ]:
#sys.path.insert(0, google_drive_packages_path)

# installing PyRosetta
if sys.version_info.major != 3 or sys.version_info.minor != 6:
    print('Need Python-3.6 to run!')
    sys.exit(1)

import_start_time = time.time()

# upload PyRosetta Linux package into your google drive and put it path here
#pyrosetta_distr_path = google_drive.replace(' ', '\\ ') + '/PyRosetta/*'

# or alternatively you can download PyRosetta directly from GrayLab web site (but this might take some time!)
!mkdir /content/PyRosetta

# HERE, change user and password with the ones obtained through the pyRosetta license
!cd /content/PyRosetta && wget -O _PyRosetta_.tar.bz2 --user levinthal --password paradox http://graylab.jhu.edu/download/PyRosetta4/archive/release/PyRosetta4.MinSizeRel.python36.linux/PyRosetta4.MinSizeRel.python36.linux.release-260.tar.bz2   
pyrosetta_distr_path = '/content' + '/PyRosetta/_PyRosetta_.tar.bz2'

!mkdir _PyRosetta_
!tar xjf /content/PyRosetta/_PyRosetta_.tar.bz2 -C _PyRosetta_ --strip-components=1
!cd _PyRosetta_/setup && python setup.py install --install-lib=$pyrosetta_install_prefix_path
!rm -rf _PyRosetta_

import_end_time = time.time()

print(f'PyRosetta setup took: {import_end_time - import_start_time:.1f}s...')

In [ ]:
for e in os.listdir(pyrosetta_install_prefix_path):
    sys.path.append(pyrosetta_install_prefix_path + '/' + e)

Finally, we will perform some tests by importing pyRosetta modules to check that everything is working smoothly.

In [ ]:
from pyrosetta import *
pyrosetta.init()

In [ ]:
import pandas as pd
from pyrosetta import *
from rosetta.protocols.rosetta_scripts import *
from pyrosetta import (
    init, pose_from_sequence, pose_from_file, Pose, MoveMap, create_score_function, get_fa_scorefxn,
    MonteCarlo, TrialMover, SwitchResidueTypeSetMover, PyJobDistributor,
)

# This notebook will be continuously updated